### 【 데이터 전처리 - 이상치 탐지 】
- 정상적인 데이터 범위를 벗어나는 값
- 잘 못 입력 또는 변환 시 오류로 발생
- 정확한 데이터 분석에 좋지 않은 데이터
- 해결 방법
    * 새롭게 데이터 수집 <--- BEST : 현실적 어려움/비용/시간
    * 데이터에 대한 처리
        - 절삭
        - 대체/치환 : 극단값으로 변경

[1] 모듈 로딩 및 데이터 준비<hr>

In [4]:
## 모듈 로딩
import numpy as np
import pandas as pd

In [5]:
## 테스트 데이터 & DF 생성
##-> 재현성 설정  -> 사용하면 랜덤값이 고정됨 
np.random.seed(0)

##-> 평균 50, 표준편차 5인 정규분포 + 일부러 큰 이상치 추가
x = np.random.normal(50, 5, size=100).tolist() + [120, 130, 5]

##-> DF 생성
df = pd.DataFrame({"score": x})

In [6]:
## 기본 정보 확인 : 앞에 5개행, 끝 5개행
display( df.head(), df.tail() )

,score
0,58.820262
1,52.000786
2,54.893690
3,61.204466
4,59.337790


,score
98,50.634560
99,52.009947
100,120.000000
101,130.000000
102,5.000000


[1] 이상치 탐지 - IQR 기반 : 가장 쉬운 고전 방식  <hr>

- 사분위수(Quartile) : 크기순으로 나열한 후 동일 비율로 4등분 할 때의 경계가 되는 세 위치의 점
    * 자료의 변동/퍼져있는 정도/변동성 척도임!
    * 제1사분위수 (Q1): 데이터의 하위 25% 지점
    * 제2사분위수 (Q2): 데이터의 중간값(중앙값) 50% 지점 
    * 제3사분위수 (Q3): 데이터의 상위 75% 지점
    * 사분위수 범위IQR: Q3 - Q1, 데이터의 중앙 50%가 모여 있는 범위
- 특징
    * 분포 측정: 데이터의 중간 50%가 얼마나 퍼져 있는지를 나타냄 => 사분위범위(IQR)
    * 이상치 감지: 이상치를 탐지하는 데 효과적
    * 분포의 유연성: 전체 범위나 표준편차와 달리 극단적인 값(이상치)의 영향 덜 받음
    * 분포의 비대칭성 파악

- 예시
    * 점수: 50, 60, 65, 70, 75, 80, 85, 90, 95
    * 최소값: 50
    * 최대값: 95
    * 중앙값/2사분위수(Q2) : 75
    * 1사분위수(Q1): 중앙값(75)을 기준으로 왼쪽(하위 절반) 데이터의 중앙값
        - 하위 절반 데이터: 50, 60, 65, 70 (짝수 4개이므로 가운데 두 값의 평균)
        - Q1 = (60+65)/2 = 62.5
    * 3사분위수(Q3): 중앙값(75)을 기준으로 오른쪽(상위 절반) 데이터의 중앙값
        - 상위 절반 데이터: 80, 85, 90, 95 (짝수 4개이므로 가운데 두 값의 평균)
        - Q3 = (85+90)/2 = 87.5



In [7]:
## ----------------------------------------------------------
## 이상치 체크 
## ----------------------------------------------------------
## - 사분위수 중 25%, 75% 값 계산 
Q1 = df["score"].quantile(0.25)
Q3 = df["score"].quantile(0.75)

## - 사분위 범위 계산/하한값/상한값 
IQR  = Q3 - Q1
low  = Q1 - 1.5 * IQR
high = Q3 + 1.5 * IQR
print(f'IQR : {IQR}, low : {low},  high : {high}')

## - 하한값/상한값 밖에 존재하는 데이터 : 이상치 
df["is_outlier_iqr"] = ~df["score"].between(low, high)
df[df["is_outlier_iqr"]].head()

IQR : 7.113276556089545, low : 36.0631288122234,  high : 64.51623503658158


,score,is_outlier_iqr
100,120.0,True
101,130.0,True
102,5.0,True


[2] 이상치 처리 <hr>

In [8]:
## ----------------------------------------------------------
## [2-1] IQR기반 이상치 제거
##       -> 표본이 충분하고 이상치가 명백할 때
## ----------------------------------------------------------
## - 하한값과 상한값 사이의 데이터만 추출
df_iqr_dropped = df[df["score"].between(low, high)].copy()

print("원본 행수:", len(df), "→ 제거 후:", len(df_iqr_dropped))


원본 행수: 103 → 제거 후: 100


In [11]:
## ----------------------------------------------------------
## [2-2] IQR기반 IQR로 값 다듬기(윈저라이즈, clip)
##       -> 극단값을 경계값으로 늘려서 보존
## ----------------------------------------------------------
df_iqr_clipped = df.copy()

## 아랫쪽 극단값 - 하한값으로 채우기 / 윗쪽 극단값 - 상한값으로 채우기
df_iqr_clipped["score"] = df_iqr_clipped["score"].clip(lower=low, upper=high)

print(df.describe())
print('----------------')
df_iqr_clipped.describe()

            score
count  103.000000
mean    51.309748
std     12.405256
min      5.000000
25%     46.733044
50%     50.608375
75%     53.846320
max    130.000000
----------------


,score
count,103.000000
mean,50.436889
std,5.552360
min,36.063129
25%,46.733044
50%,50.608375
75%,53.846320
max,64.516235


In [ ]:
## ----------------------------------------------------------
## [2-3] Z-score로 이상치 제거 
##       -> 데이터의 정규분포 가정 
##       -> 평균±3표준편차 밖을 이상치로 보는 흔한 방식
##       -> Outlier에 영향 큼
## ----------------------------------------------------------
## 평균/중앙값/최빈값 계산 
mean_z = df["score"].mean()
median_z = df["score"].median()    # 모집단 표준편차 기준
mode_z = df["score"].mode() 
print(f'{mean_z}, {median_z}, {mode_z}')


## - 평균, 표준편차 계산 z-score = (x - 평균)/표준편차
mu = df["score"].mean()
sd = df["score"].std(ddof=0)     # 모집단 표준편차 기준
z = (df["score"] - mu) / sd


# 임계값: 보통 |z| > 3 사용(필요에 따라 2.5, 3.5 등 조정)
df['zscore']    = z             # zscore 컬럼추가
df['outlier_Z'] = z.abs() > 3   # boolean indexing 결과 컬럼추가

## - 평균/표준편차/이상치 행 출력
print("평균/표준편차:", mu, sd)
display(df[df['outlier_Z']])

## - 이상치 제외한 나머지 행 추출
df_z_dropped = df[(z >= -3) & (z <= 3)].copy()
len(df), len(df_z_dropped)

51.30974764822567, 50.60837508246414, 0        5.000000
1       37.235051
2       40.096018
3       41.368587
4       41.468649
          ...    
98      59.753877
99      61.204466
100     61.348773
101    120.000000
102    130.000000
Name: score, Length: 103, dtype: float64
평균/표준편차: 51.30974764822567 12.34488960652997


,score,is_outlier_iqr,zscore,outlier_Z
100,120.0,True,5.564266,True
101,130.0,True,6.374318,True
102,5.0,True,-3.751329,True


(103, 100)

In [ ]:
## ----------------------------------------------------------
## [2-4] 백분위수(Percentile) 클리핑
##       -> 백분위수 = 데이터를 0~100%로 나눈 누적 분포 상의 위치
##       -> 하위 p%, 상위 (100−p)% 값을 경계로 잘라냄
##       -> 단순/빠르며 이상치에 민감도 완화
##       -> 데이터 양 적으면 경계 불안정   
##       -> 자주 사용되는 기중 : (1%, 99%) 또는 (2.5%, 97.5%)   
##  ★ 데이터 분포를 안정화시키는, 가장 간단하고 안전한 이상치 완화 기법
## ----------------------------------------------------------
## - 하위 1% / 상위 99% 구간 밖 값들을 잘라냄(clip)
p01 = df["score"].quantile(0.01)
p99 = df["score"].quantile(0.99)

df_pct_clipped = df.copy()
df_pct_clipped["score"] = df_pct_clipped["score"].clip(p01, p99)

print('이상치 처리 전', df.describe())

print('이상치 처리 후', df_pct_clipped.describe())


이상치 처리 전             score        zscore
count  103.000000  1.030000e+02
mean    51.309748 -9.140477e-16
std     12.405256  1.004890e+00
min      5.000000 -3.751329e+00
25%     46.733044 -3.707367e-01
50%     50.608375 -5.681481e-02
75%     53.846320  2.054755e-01
max    130.000000  6.374318e+00
이상치 처리 후             score        zscore
count  103.000000  1.030000e+02
mean    51.503956 -9.140477e-16
std     10.823843  1.004890e+00
min     37.292270 -3.751329e+00
25%     46.733044 -3.707367e-01
50%     50.608375 -5.681481e-02
75%     53.846320  2.054755e-01
max    118.826975  6.374318e+00


In [ ]:
## ----------------------------------------------------------
## [2-5] MAD(중앙절대편차) 기반 
##       -> 이상치에 강한(강건한) 방법
##       -> Z-score 공식을 중앙값 기준으로 바꾼 버전
##       -> 극단값이 많아도 중앙값/중앙편차를 쓰므로 안정적
##       -> MAD = median( |x - median(x)| )
##       -> 0.6745 = 보정 상수 (정규분포일 때 표준편차와 스케일 맞추는 역할)
## ----------------------------------------------------------
## - 하위 1% / 상위 99% 구간 밖 값들을 잘라냄(clip)  - 중앙값/각 원소-중앙값 계산
med = df["score"].median()
mad = (df["score"].sub(med)).abs().median()

## - Robust Z-like score (0.6745로 정규성 보정)
## - Z-score 공식을 중앙값 기준으로 바꾼 버전
## - 중앙값(Median) 과 중앙절대편차(MAD)를 사용
## - 공식 :0.6745×(x−median)/MAD 
## -                mad 
## -  ┌───────────── 중앙값 ─────────────┐
## -  |                |                 |
## -  낮음        정상 범위(IQR내)     높음
## -       ↑             ↑
## -        Robust Z=−3.5    Robust Z=+3.5  ← 이상치 경계선
robust_z = 0.6745 * (df['score'] - med) / (mad if mad != 0 else 1)

df['robust_z']    = robust_z                ## - robust_score 컬럼 추가
df['outlier_MAD'] = robust_z.abs() > 3.5    ## - robust_score 이상치 결과 True/False 컬럼 추가

print("중앙값/MAD:", med, mad)
display(df[df['outlier_MAD']])

중앙값/MAD: 50.60837508246414 3.713805911833383


,score,is_outlier_iqr,zscore,outlier_Z,robust_z,outlier_MAD
100,120.0,True,5.564266,True,12.602880,True
101,130.0,True,6.374318,True,14.419076,True
102,5.0,True,-3.751329,True,-8.283376,True


In [22]:
## ----------------------------------------------------------
## [2-6] 범주별로 처리
##       -> 각 그룹마다 분포가 다르면 그룹별 IQR로 처리
## ----------------------------------------------------------

##-> 범주열 추가
df_g = df.copy()
df_g["group"] = np.where(df_g.index % 2 == 0, "A", "B")

##-> 그룹별 클리핑 처리 함수 
def group_iqr_clip(df, col="score", k=1.5):
    q1, q3 = df[col].quantile([0.25, 0.75])
    iqr = q3 - q1
    low, high = q1 - k*iqr, q3 + k*iqr
    df[col] = df[col].clip(low, high)
    return df

##-> 그룹화 후 클리핑 진행
df_group         = df_g.groupby("group", group_keys=False)
df_group_clipped = df_group.apply(group_iqr_clip, col="score", include_groups=False)
df_group_clipped.head()

,score,is_outlier_iqr,zscore,outlier_Z
0,58.820262,False,0.608391,False
1,52.000786,False,0.055978,False
2,54.893690,False,0.290318,False
3,61.204466,False,0.801523,False
4,59.337790,False,0.650313,False


In [23]:
## ----------------------------------------------------------
## [2-7] 여러 수치열에 한 번에 적용: IQR 클리핑
## ----------------------------------------------------------
def iqr_clip(sr, k: float = 1.5):
    q1, q3 = sr.quantile([0.25, 0.75])
    iqr = q3 - q1
    low, high = q1 - k*iqr, q3 + k*iqr
    return sr.clip(low, high)

#  여러 수치열이 있을 때
df_multi = pd.DataFrame({
    "x": np.random.normal(50, 5, 103),
    "y": np.random.normal(100, 10, 103)
})

# 이상치 삽입
df_multi.loc[[0,1], "x"] = [150, -10]
df_multi.loc[[2,3], "y"] = [300, -50]

df_multi_clipped = df_multi.apply(iqr_clip)
df_multi_clipped.describe()

,x,y
count,103.000000,103.000000
mean,50.401235,99.545249
std,5.499641,10.010888
min,34.343538,75.743973
25%,46.289205,93.872982
50%,50.087396,99.018496
75%,54.252984,105.958988
max,66.198652,124.087996
